<a href="https://colab.research.google.com/github/rehanbchinoy/Math-156-project/blob/main/ulysses_remake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [3]:
#read the txt file to str

from google.colab import files
uploaded = files.upload()

Saving ulysses_james_joyce.txt to ulysses_james_joyce.txt


In [32]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ulysses = open('ulysses_james_joyce.txt', 'rb').read().decode(encoding='utf-8')
len(ulysses)

1571207

In [5]:
#unique characters in the file
vocab = sorted(set(ulysses))
print(f'{len(vocab)} unique characters')

124 unique characters


In [6]:
#all the characters appeared in Ulysses
#vocab

## Process the Text

### Vectorize

In [7]:
#function that converts character tokens to numeric ids
ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

In [8]:
#function that converts numeric ids to character tokens
chars_from_ids = preprocessing.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [9]:
#function that combines character tokens back to strings
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### Creating training examples and targets

In [10]:
#convert ulysses from strings to numeric vector
all_ids = ids_from_chars(tf.strings.unicode_split(ulysses, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1571207,), dtype=int64, numpy=array([117,   3,  39, ...,   1,   2,   1])>

In [11]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [12]:
#verify the ids dataset is correct
for ids in ids_dataset.take(100):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

—
 
I
 
—









[
 
1
 
]






S
t
a
t
e
l
y
,
 
p
l
u
m
p
 
B
u
c
k
 
M
u
l
l
i
g
a
n
 
c
a
m
e
 
f
r
o
m
 
t
h
e
 
s
t
a
i
r
h
e
a
d
,
 
b
e
a
r
i
n
g
 
a
 
b
o
w
l
 
o
f



l
a
t
h
e
r
 


In [13]:
#split the text into sequences with length 100
seq_length = 100
examples_per_epoch = len(ulysses)//(seq_length+1)

In [14]:
#create the sequences based on sequences with length 101
#(it's 101 bc we need to split it into training and target sequences)
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

In [15]:
#verify the sequences are created correctly
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xe2\x80\x94' b' ' b'I' b' ' b'\xe2\x80\x94' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'[' b' ' b'1' b' ' b']' b'\r' b'\n' b'\r' b'\n' b'S' b't'
 b'a' b't' b'e' b'l' b'y' b',' b' ' b'p' b'l' b'u' b'm' b'p' b' ' b'B'
 b'u' b'c' b'k' b' ' b'M' b'u' b'l' b'l' b'i' b'g' b'a' b'n' b' ' b'c'
 b'a' b'm' b'e' b' ' b'f' b'r' b'o' b'm' b' ' b't' b'h' b'e' b' ' b's'
 b't' b'a' b'i' b'r' b'h' b'e' b'a' b'd' b',' b' ' b'b' b'e' b'a' b'r'
 b'i' b'n' b'g' b' ' b'a' b' ' b'b' b'o' b'w' b'l' b' ' b'o' b'f' b'\r'
 b'\n' b'l' b'a' b't' b'h' b'e' b'r' b' ' b'o'], shape=(101,), dtype=string)


In [16]:
#generate the target sequence for each sequence
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
dataset = sequences.map(split_input_target)
dataset

<MapDataset shapes: ((100,), (100,)), types: (tf.int64, tf.int64)>

### Create Training Batches

In [18]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Build The Model

In [19]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [20]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)
 
  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [21]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try The Model

Run the model to see if the output is as expected

In [22]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 125) # (batch_size, sequence_length, vocab_size)


In [23]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  32000     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  128125    
                                                                 
Total params: 4,098,429
Trainable params: 4,098,429
Non-trainable params: 0
_________________________________________________________________


In [24]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [25]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'the huge high door of the Irish house of parliament a flock of\r\npigeons flew. Their little frolic af'

Next Char Predictions:
 b'y\ni+w\xc3\xa8x\xe2\x80\xa2_;)Fz5Q\xc3\xbc!VZA\xc3\x9c\no\xc5\x93\xe2\x9c\xa0\xe2\x80\x9c\xc5\x93 VekU9\xe2\x9c\xa0On$\xc3\xa83\n,s,\xc3\xb9M3\xc3\x87ko\xc3\xa7IbhHTqFv_W\xc3\xba5%bS\xc3\xbbnenx$\xe2\x80\x9drd0L\xc3\x9cbdI;Ze\xc4\x81K\xc2\xb0!hei[e\xc3\xa1H\xe2\x80\xa6/\xc3\xa9\xc3\xaa\xc3\xa8t'


## Train The Model

### Add optimizer and loss

In [26]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [27]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 125)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.827674


Check to make sure the loss is roughly the same as the vocabulary size

In [28]:
tf.exp(mean_loss).numpy()

124.920044

In [29]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [52]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks'
# Name of the checkpoint files
#checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_dir,
    save_weights_only=True)

### Execute the training

In [47]:
EPOCHS = 20

In [48]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7953
Epoch 2/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7792
Epoch 3/20
243/243 [==============================] - 33s 128ms/step - loss: 0.7654
Epoch 4/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7529
Epoch 5/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7410
Epoch 6/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7340
Epoch 7/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7287
Epoch 8/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7220
Epoch 9/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7191
Epoch 10/20
243/243 [==============================] - 33s 127ms/step - loss: 0.7190
Epoch 11/20
243/243 [==============================] - 32s 127ms/step - loss: 0.7168
Epoch 12/20
243/243 [==============================] - 33s 128ms/step - lo

## Generate Text

In [49]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [50]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [51]:
start = time.time()
states = None
next_char = tf.constant(['Duke'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

Duke!

CISSY CAFFREY: _(Shakes hair before him.)_ You couldn’t show. _(She knew then he stood
out.)_

THE CROST ARE TATCONS: _(This indeeds, his twoslinent pages for copies excited the
modelatile or every cates, breasts, then gay, among the
Phieldo!_

Sufford and ouncelorred from France. Sadly the babemadination of hymenon (cestar) which
was _Synn’s Venum)_ which was insistence of modering dog he could
feel me to wait. The weather is clear then the stone for your mother
thats be to wodding after friends from the pelvical breast: eyedark strawings out of
inside, richeded irly responsible from Winner street,
then by the ladder and elevant: passionate gentiles, by sappier about tonight. Itwell in answer
with a heartbedy. Now who is this old part? May Mirante that Michael Gurl._

BEW’S Days eyebelling out their damned kalk I suppose, says Joe.

—What’s that? Kinch, thoughts contributor, a word figure
worthy preparation of a belladone only to think of the union with the out  

_____________